In [1]:
## import

import notears.utils as ut
import torch
import numpy as np
import os
from numpy import genfromtxt
import pandas as pd
import graphviz
import lingam
from lingam.utils import make_prior_knowledge, make_dot
import warnings
warnings.filterwarnings("ignore")
import igraph as ig
from sklearn.preprocessing import StandardScaler

In [2]:
## environmental setup

print([np.__version__, pd.__version__, graphviz.__version__, lingam.__version__])
torch.set_default_dtype(torch.double)
np.set_printoptions(precision=3, suppress=True)
np.random.seed(0)

['1.21.5', '1.3.4', '0.20.1', '1.6.0']


In [3]:
## functions

def make_prior_knowledge_graph(prior_knowledge_matrix):
    d = graphviz.Digraph(engine='dot')

    labels = [f'x{i}' for i in range(prior_knowledge_matrix.shape[0])]
    for label in labels:
        d.node(label, label)

    dirs = np.where(prior_knowledge_matrix > 0)
    for to, from_ in zip(dirs[0], dirs[1]):
        d.edge(labels[from_], labels[to])

    dirs = np.where(prior_knowledge_matrix < 0)
    for to, from_ in zip(dirs[0], dirs[1]):
        if to != from_:
            d.edge(labels[from_], labels[to], style='dashed')
    return d

In [4]:
## variables

list_dt = ['linear'] ## ['nonlinear', 'linear']
list_n = [200, 1000] ## [200, 1000]
list_d = [10, 20] ## [10, 20]
list_s0_factor = [1, 4] ## [1, 4]
list_gt = ['ER', 'SF'] ## ['ER', 'SF']
list_st = []
for dt in list_dt:
    if dt == 'linear':
        list_st = list_st + ['gauss']
    elif dt == 'nonlinear':
        list_st = list_st + ['mim']
    else:
        raise Exception('data type not valid!')
list_should_std = [False, True] ## [False, True]
n_trials = 10 ## 10
max_iter = 10

In [5]:
## experiments

d_result = {}
for dt in list_dt:
    for n in list_n:
        for d in list_d:
            for s0_factor in list_s0_factor:
                for gt in list_gt:
                    for st in list_st:
                        for should_std in list_should_std:
                            for trial_no in range(n_trials):
                                ut.set_random_seed(123+trial_no)

                                ## part 1: generate causal graph
                                s0 = d * s0_factor
                                B_true = ut.simulate_dag(d, s0, gt)
                                folder_name = str(dt) + '_n_d_s0_gt_sem_' \
                                                + str(n) + '_' + str(d) + '_' \
                                                    + str(s0) + '_' + str(gt) + '_' + str(st)
                                folder_path = 'datasets/' + folder_name + '/'
                                if os.path.exists(folder_path):
                                    pass 
                                else:
                                    os.makedirs(folder_path)
                                file_name = str(trial_no) + '_W_true.csv'
                                file_path = folder_path + file_name
                                if os.path.exists(file_path):
                                    B_true = genfromtxt(file_path, delimiter=',')
                                else:                                
                                    np.savetxt(file_path, B_true, delimiter=',')
                                #################################################
                                
                                ## part 2: generate data
                                if dt == 'nonlinear':
                                    X = ut.simulate_nonlinear_sem(B_true, n, st)
                                elif dt == 'linear':
                                    W_true = ut.simulate_parameter(B_true)                                  
                                    X = ut.simulate_linear_sem(W_true, n, st)
                                else:
                                    raise Exception('data type not valid!')
                                file_name = str(trial_no) + '_X.csv'
                                file_path = folder_path + file_name
                                if os.path.exists(file_path):
                                    X = genfromtxt(file_path, delimiter=',')
                                else:                                
                                    np.savetxt(file_path, X, delimiter=',')
                                #################################################

                                ## part 3: experiment with augmented causal learning
                                mask = np.full((d, d), -1)
                                model = lingam.DirectLiNGAM(prior_knowledge=mask)
                                if should_std:
                                    scaler = StandardScaler().fit(X)
                                    X = scaler.transform(X)
                                model.fit(X)
                                B_pred = model.adjacency_matrix_.T 
                                ## donot use model.adjacency_matrix_ to compare with B_true
                                ## use model.adjacency_matrix_.T
                                ## because for notears W_{i,j} indicates i -> j (from i to j)
                                ## but for DirectLingam W_{i,j} indicates j -> i (from j to i)
                                file_name = str(trial_no) + '_W_est.csv'
                                file_path = folder_path + file_name                                
                                np.savetxt(file_path, B_pred, delimiter=',')
                                acc = ut.count_accuracy(B_true, B_pred != 0)
                                fdr, tpr, fpr, shd, nnz = acc['fdr'], acc['tpr'], acc['fpr'], acc['shd'], acc['nnz'] 
                                k = (dt, n, d, s0, gt, st, should_std, trial_no, 'init')
                                v = (fdr, tpr, fpr, shd, nnz)
                                d_result[k] = v
                                
                                ## terminate:= if (mask == mask_next) or (iter_no == 10)
                                ##
                                mask_pred = np.where(model.adjacency_matrix_ == 0, 0, 1) 
                                mask_true = B_true.T 
                                mask_isect = np.full((d, d), -1)
                                for i in range(d):
                                    for j in range(d):
                                        if mask_pred[i][j] == mask_true[i][j]:
                                            mask_isect[i][j] = mask_pred[i][j]
                                mask_next = mask_isect
                                ##
                                iter_no = 0
                                while ((not np.array_equal(mask, mask_next)) and (iter_no < max_iter)):
                                    mask = mask_next
                                    
                                    ########
                                    model = lingam.DirectLiNGAM(prior_knowledge=mask)
                                    try:
                                        model.fit(X)
                                    except Exception as e:
                                        print(e)
                                        print('Breaking while loop: ', dt, n, d, s0, gt, st, should_std, trial_no, iter_no)
                                        break

                                    B_pred = model.adjacency_matrix_.T 
                                    file_name = str(trial_no) + '_W_est_final.csv'
                                    file_path = folder_path + file_name                                
                                    np.savetxt(file_path, B_pred, delimiter=',')
                                    acc = ut.count_accuracy(B_true, B_pred != 0)
                                    fdr, tpr, fpr, shd, nnz = acc['fdr'], acc['tpr'], acc['fpr'], acc['shd'], acc['nnz'] 
                                    k = (dt, n, d, s0, gt, st, should_std, trial_no, 'final')
                                    v = (fdr, tpr, fpr, shd, nnz)
                                    d_result[k] = v
                                    ########
                                    ##
                                    mask_pred = np.where(model.adjacency_matrix_ == 0, 0, 1)
                                    mask_true = B_true.T 
                                    mask_isect = np.full((d, d), -1)
                                    for i in range(d):
                                        for j in range(d):
                                            if mask_pred[i][j] == mask_true[i][j]:
                                                mask_isect[i][j] = mask_pred[i][j]
                                    mask_next = mask_isect
                                    ##
                                    iter_no+=1
                                    
                                    ##DELETE                                    
                                    print(iter_no, np.array_equal(mask, mask_next))
                                    ##DELETE   

In [7]:
## evaluation 

## d_result
## for each type of combination:
##     check fdr(mean+std)(init X final), 
##     check tpr(mean+std)(init X final), 
##     check fpr(mean+std)(init X final), 
##     check shd(mean+std)(init X final), 
list_result = []
for dt in list_dt:
    for n in list_n:
        for d in list_d:
            for s0_factor in list_s0_factor:
                for gt in list_gt:
                    for st in list_st:
                        for should_std in list_should_std:
                            s0 = d * s0_factor                            
                            
                            list_init_fdr = []
                            list_final_fdr = []
                            
                            list_init_tpr = []
                            list_final_tpr = []
                            
                            list_init_fpr = []
                            list_final_fpr = []
                            
                            list_init_shd = []
                            list_final_shd = []
                            
                            for trial_no in range(n_trials):
                                k1 = (dt, n, d, s0, gt, st, should_std, trial_no, 'init')
                                v1 = d_result[k1]
                                init_fdr, init_tpr, init_fpr, init_shd = v1[0], v1[1], v1[2], v1[3]
                                list_init_fdr.append(init_fdr)
                                list_init_tpr.append(init_tpr)
                                list_init_fpr.append(init_fpr)
                                list_init_shd.append(init_shd)
                                
                                k2 = (dt, n, d, s0, gt, st, should_std, trial_no, 'final')
                                try:
                                    v2 = d_result[k2]
                                    final_fdr, final_tpr, final_fpr, final_shd = v2[0], v2[1], v2[2], v2[3]
                                    list_final_fdr.append(final_fdr)
                                    list_final_tpr.append(final_tpr)
                                    list_final_fpr.append(final_fpr)
                                    list_final_shd.append(final_shd)                                    
                                except Exception as e:
                                    print(e)
                                    print('key error: ', n, d, s0, gt, st, should_std, trial_no)

                            mean_init_fdr, std_init_fdr = np.mean(list_init_fdr), np.std(list_init_fdr)
                            mean_final_fdr, std_final_fdr = np.mean(list_final_fdr), np.std(list_final_fdr)
                            mean_init_tpr, std_init_tpr = np.mean(list_init_tpr), np.std(list_init_tpr)
                            mean_final_tpr, std_final_tpr = np.mean(list_final_tpr), np.std(list_final_tpr)                            
                            mean_init_fpr, std_init_fpr = np.mean(list_init_fpr), np.std(list_init_fpr)
                            mean_final_fpr, std_final_fpr = np.mean(list_final_fpr), np.std(list_final_fpr)                           
                            mean_init_shd, std_init_shd = np.mean(list_init_shd), np.std(list_init_shd)
                            mean_final_shd, std_final_shd = np.mean(list_final_shd), np.std(list_final_shd)  
 
                            # print(dt, n, d, s0_factor*d, gt, st, should_std)
                            # print('fdr: ', mean_init_fdr, '±', std_init_fdr, mean_final_fdr, '±', std_final_fdr)                            
                            # print('tpr: ', mean_init_tpr, '±', std_init_tpr, mean_final_tpr, '±', std_final_tpr)                            
                            # print('fpr: ', mean_init_fpr, '±', std_init_fpr, mean_final_fpr, '±', std_final_fpr)                            
                            # print('shd: ', mean_init_shd, '±', std_init_shd, mean_final_shd, '±', std_final_shd)                            
                            # print()
                            # print()
                            
                            result = [
                                str(n), str(d), str(s0_factor*d), str(gt), str(should_std),
                                '{:.2f}'.format(mean_init_fdr) + ' ± ' + '{:.2f}'.format(std_init_fdr), 
                                '{:.2f}'.format(mean_final_fdr) + ' ± ' + '{:.2f}'.format(std_final_fdr), 
                                '{:.2f}'.format(mean_init_tpr) + ' ± ' + '{:.2f}'.format(std_init_tpr), 
                                '{:.2f}'.format(mean_final_tpr) + ' ± ' + '{:.2f}'.format(std_final_tpr), 
                                '{:.2f}'.format(mean_init_fpr) + ' ± ' + '{:.2f}'.format(std_init_fpr), 
                                '{:.2f}'.format(mean_final_fpr) + ' ± ' + '{:.2f}'.format(std_final_fpr), 
                                '{:.2f}'.format(mean_init_shd) + ' ± ' + '{:.2f}'.format(std_init_shd), 
                                '{:.2f}'.format(mean_final_shd) + ' ± ' + '{:.2f}'.format(std_final_shd), 

                            ]
                            list_result.append(result)
                            

('linear', 200, 10, 10, 'SF', 'gauss', False, 6, 'final')
key error:  200 10 10 SF gauss False 6
('linear', 200, 10, 10, 'SF', 'gauss', True, 6, 'final')
key error:  200 10 10 SF gauss True 6
('linear', 200, 10, 40, 'ER', 'gauss', False, 1, 'final')
key error:  200 10 40 ER gauss False 1
('linear', 200, 10, 40, 'ER', 'gauss', False, 5, 'final')
key error:  200 10 40 ER gauss False 5
('linear', 200, 10, 40, 'ER', 'gauss', True, 1, 'final')
key error:  200 10 40 ER gauss True 1
('linear', 200, 10, 40, 'SF', 'gauss', False, 9, 'final')
key error:  200 10 40 SF gauss False 9
('linear', 200, 10, 40, 'SF', 'gauss', True, 9, 'final')
key error:  200 10 40 SF gauss True 9
('linear', 200, 20, 20, 'ER', 'gauss', False, 2, 'final')
key error:  200 20 20 ER gauss False 2
('linear', 200, 20, 20, 'ER', 'gauss', False, 9, 'final')
key error:  200 20 20 ER gauss False 9
('linear', 200, 20, 20, 'ER', 'gauss', True, 2, 'final')
key error:  200 20 20 ER gauss True 2
('linear', 200, 20, 20, 'ER', 'gauss',

In [8]:
## generate result 

## convert your array into a dataframe
array_result = np.array(list_result)
df_result = pd.DataFrame (array_result)

## save to xlsx file
filepath = 'datasets/script_1_result_2.xlsx'
df_result.to_excel(filepath, index=False)